In [ ]:
#%pip install requests

In [4]:
import requests
import json
from pathlib import Path
import sys

# Define Overpass API endpoint and the Query
overpass_url = "https://overpass-api.de/api/interpreter"


overpass_query = """
[out:json][timeout:180];
/* 1. Get the Berlin boundary by its Area ID (Relation ID 62422) */
area(3600062422)->.berlin;

/* 2. Find all nightclubs inside that boundary */
(
  node["amenity"="nightclub"](area.berlin);
  way["amenity"="nightclub"](area.berlin);
  relation["amenity"="nightclub"](area.berlin);
);

/* 3. Output the results */
out body;
>;
out skel qt;
"""

#  Define the output file path 
output_dir = Path('../source')
output_dir.mkdir(parents=True, exist_ok=True) # Ensure 'source' folder exists
output_geojson_file = output_dir / 'night_clubs_raw.geojson'

# Send the request to Overpass API 
print("Sending query to Overpass API... (this may take up to 180 seconds)")
try:
    response = requests.post(overpass_url, data=overpass_query)
    
    # Check if the request was successful
    response.raise_for_status() 
    print("Query successful. Data received.")
    
    # Get the JSON data from the response
    # The API returns GeoJSON directly if queried correctly
    data = response.json()

    # Save the raw JSON data to a .geojson file 
    with open(output_geojson_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Raw GeoJSON data successfully saved to: {output_geojson_file}")

except requests.exceptions.RequestException as e:
    print(f"❌ An error occurred connecting to Overpass API: {e}")
    # Print server error message if available
    if e.response:
        print(f"Server response:\n{e.response.text}")
    sys.exit(1) # Exit the script if the download failed

except json.JSONDecodeError:
    print("❌ Failed to decode JSON from the response. The API might be down or the query is wrong.")
    print(f"Server response:\n{response.text}")
    sys.exit(1)

except Exception as e:
    print(f"An unexpected error occurred: {e}")
    sys.exit(1)

Sending query to Overpass API... (this may take up to 180 seconds)
Query successful. Data received.
✅ Raw GeoJSON data successfully saved to: ..\source\night_clubs_raw.geojson
